In [2]:
!pip install deepchem
!pip install transformers
!pip install openbabel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 693 kB 6.7 MB/s 
     |████████████████████████████████| 29.3 MB 230 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 7.5 MB/s 
     |████████████████████████████████| 182 kB 78.8 MB/s 
     |████████████████████████████████| 7.6 MB 56.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 82 kB 1.2 MB/s 
  ERROR: Failed building wheel for openbabel
  Running setup.py clean for openbabel
Failed to build openbabel
    Running setup.py install for openbabel ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-kj98zc9o/openbabel_6afc9f1e076042a1ad4228b99cab

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import string
import torch.nn as nn
from deepchem.feat.smiles_tokenizer import SmilesTokenizer
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
# from openbabel import pybel

In [4]:
from google.colab import drive
import sys
import os

drive.mount('/content/drive')
sys.path.insert(0,'/content/drive/My Drive/Fall 2022/CS 137/Final Project/models')
os.chdir('/content/drive/My Drive/Fall 2022/CS 137/Final Project/models')

Mounted at /content/drive


In [5]:
datapath = 'full_data.txt'
vocabpath = 'vocab.txt'

In [6]:
datafile = pd.read_csv(datapath, index_col=False)#.drop('Unnamed: 0')
datafile.drop(columns=datafile.columns[0], axis=1, inplace=True)
datafile

,atoms,smiles,CSD_code,q,S,Stoichiometry,MND,Electronic_E,Dispersion_E,Dipole_M,Metal_q,HL_Gap,HOMO_Energy,LUMO_Energy,Polarizability
0,116,[La]1(N(c2c(cccc2C(C)C)C(C)C)[C](N1c1c(cccc1C(...,WIXKOE,0,0,C47H65LaN2O,7,-2045.524942,-0.239239,4.2333,2.10934,0.13108,-0.16204,-0.03096,598.457913
1,42,[Sc]123(ON(O1)[O])(ON([O])O2)ON([O])O3.n1ccccc...,DUCVIG,0,0,C13H10N9O9Sc,8,-2430.690317,-0.082134,11.7544,0.75994,0.12493,-0.24358,-0.11865,277.750698
2,73,[La]123(O[C](C=C(O1)C(F)(F)F)C(F)(F)F)(OC(=C[C...,KINJOG,0,0,C23H21F18LaO10,10,-3467.923206,-0.137954,8.3017,1.76650,0.14014,-0.23646,-0.09632,393.442545
3,46,[Sc]123(OC(=C[C](O1)C(F)(F)F)C(F)(F)F)(O[C](C(...,GEKYEC,0,0,C15H5F18O7Sc,7,-3657.137747,-0.073924,3.0448,1.17186,0.13865,-0.26765,-0.12900,266.725736
4,72,n1([Y][C@@]2(C(=C(C(=C2C)C)C)C)C)c(c(c(c1C)C)C...,PIBNEV,0,0,C28H42NY,11,-1184.911899,-0.132369,2.7760,1.92642,0.10641,-0.15164,-0.04523,342.341585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86660,56,[Zn]12(Cl)(Cl)N3[CH]C=CC=C3C[N@@]1(C[C]1N2C=CC...,XEKGOL,0,0,C24H23Cl2N5OZn,5,-3976.162059,-0.107885,8.8712,0.95959,0.13420,-0.21155,-0.07735,348.434765
86661,63,[Cd]123(S[C](S1)N(C)CCCC)(S[C](S2)N(C)CCCC)N1[...,ULAWIO,0,0,C22H32CdN4S4,6,-2836.193928,-0.117090,6.3455,0.83479,0.07759,-0.16138,-0.08379,384.879049
86662,53,C12=C(c3c4c1cccc4ccc3)N(c1ccc(cc1)SC)[Zn](N2c1...,VUJJAN,0,0,C26H20Cl2N2S2Zn,4,-4607.327195,-0.107826,10.6032,0.92167,0.06897,-0.19997,-0.13100,377.567353
86663,45,[Hg](Cl)[C@@]1([Si](C)(C)C(C)(C)C)C(=C(C(=C1C)...,NEWZEU,0,0,C15H27ClHgSi,2,-1490.551341,-0.079913,4.4533,0.75218,0.09186,-0.19018,-0.09832,240.757590


# Tokenize SMILES

In [7]:
vocab = []
with open(vocabpath) as f:
    for line in f:
        vocab.append(line.rstrip())

In [8]:
def convert_smiles_to_data(smiles, tokenizer):
    """ Convert a list of strings to a list of numpy arrays"""
    data = [None] * len(smiles)
    for i, string in enumerate(smiles):
        data[i] = tokenizer.encode(string)
        
    return data

In [9]:
smiles = list(datafile['smiles'])
tokenizer = SmilesTokenizer(vocabpath)
tokenizer.encode(smiles[0])
encoded = convert_smiles_to_data(smiles, tokenizer)
datafile['encoded-smiles'] = encoded

In [10]:
sizes = []
for a in encoded:
    sizes.append(len(a))

print('Data statistics:')
print('Number of Compounds: ', len(encoded))
print('Maximum and minimum SMILES lengths:', max(sizes), ',', min(sizes))
print('Total number of characters:', sum(sizes))
print('Vocabulary size: ', len(vocab))

sample = ''
for i in encoded[0]:
    sample +=vocab[i] 
print('Random Example SMILES:', sample)
print('Random Encoded SMILES:', encoded[0])

unique = len({x for l in encoded for x in l})
print('Unique Characters:', unique)

Data statistics:
Number of Compounds:  86665
Maximum and minimum SMILES lengths: 512 , 7
Total number of characters: 5763765
Vocabulary size:  591
Random Example SMILES: [CLS][La]1(N(c2c(cccc2C(C)C)C(C)C)[C](N1c1c(cccc1C(C)C)C(C)C)C(C)(C)C)(Cc1ccccc1)Cc1ccccc1.O1CCCC1[SEP]
Random Encoded SMILES: [12, 239, 20, 17, 23, 17, 15, 21, 15, 17, 15, 15, 15, 15, 21, 16, 17, 16, 18, 16, 18, 16, 17, 16, 18, 16, 18, 94, 17, 23, 20, 15, 20, 15, 17, 15, 15, 15, 15, 20, 16, 17, 16, 18, 16, 18, 16, 17, 16, 18, 16, 18, 16, 17, 16, 18, 17, 16, 18, 16, 18, 17, 16, 15, 20, 15, 15, 15, 15, 15, 20, 18, 16, 15, 20, 15, 15, 15, 15, 15, 20, 24, 19, 20, 16, 16, 16, 16, 20, 13]
Unique Characters: 146


In [11]:
x = list(datafile['encoded-smiles'])
for i, smiles in enumerate(x):
    x[i] = torch.tensor(smiles, dtype=torch.float32)
x = pad_sequence(x, batch_first=True, padding_value=-1)
y = datafile.drop(columns=['encoded-smiles', 'smiles', 'CSD_code', 'Stoichiometry']).to_numpy()

In [12]:
class ModelData(Dataset):
    def __init__(self, x, y):
        
        self.x = torch.tensor(x, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32)
        
    def __len__(self):
#         print(self.x.size()[0])
        return self.x.size()[0]
    
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx] 
    
d = ModelData(x, y)

<ipython-input-12-dbd0352cdd9b>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x = torch.tensor(x, dtype=torch.float32)


In [13]:
class GRUNetwork(nn.Module):
    def __init__(self):
        super(GRUNetwork, self).__init__()
        torch.manual_seed(12345)
#         self.embed = nn.Embedding(num_embeddings=len(vocab), embedding_dim=32)
        self.GRU = nn.GRU(input_size=max(sizes), hidden_size=512, num_layers=3, batch_first=True, bidirectional=False, dropout=0.15)
        self.linear1 = nn.Linear(in_features=512, out_features=128)
        self.linear2 = nn.Linear(in_features=128, out_features=12)
        
    def forward(self, x):
#         x = self.embed(x)
        h, state = self.GRU(x)
        h = self.linear1(h)
        out = self.linear2(h)
        
        return out, state
    
model = GRUNetwork()

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, shuffle=True)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.8, shuffle=True)
train_set = ModelData(x_train, y_train)
val_set = ModelData(x_val, y_val)
test_set = ModelData(x_test, y_test)

<ipython-input-12-dbd0352cdd9b>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x = torch.tensor(x, dtype=torch.float32)


In [15]:
x_val.shape

torch.Size([13867, 512])

In [16]:
def train(model, train_set, val_set, reg_weight=1, num_epochs=10, batch_size=10, lr=0.02):
    
    train_loader = DataLoader(train_set, batch_size=batch_size)
    val_loader = DataLoader(val_set, batch_size=batch_size)
    
    loss_func = nn.MSELoss()
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    
    history = {"train_acc": [], "train_loss": [], "val_loss": [], "val_acc": []}
    
    for i in range(num_epochs):
        epoch_train_loss = []
        epoch_val_loss = []
        epoch_train_acc = []
        epoch_val_acc = []
        for _, (x, y) in enumerate(train_loader):
            model.train()
    

            y_hat, state = model(x)
            loss = loss_func(y_hat, y) * reg_weight

            opt.zero_grad()
            loss.backward()

            opt.step()

            train_acc = mean_squared_error(y, y_hat.detach().numpy())
            epoch_train_loss.append(loss.detach().numpy())
            epoch_train_acc.append(train_acc)
            
        with torch.no_grad():
            for _, (val_x, val_y) in enumerate(val_loader):
                val_pred, _ = model(val_x)
                val_loss = loss_func(val_pred, val_y) * reg_weight
                
                val_acc = mean_squared_error(val_y, val_pred)
                epoch_val_loss.append(val_loss.detach().numpy())
                epoch_val_acc.append(val_acc)
        
        epoch_train_loss = sum(epoch_train_loss)/len(epoch_train_loss)
        epoch_val_loss = sum(epoch_val_loss)/len(epoch_val_loss)
        epoch_val_acc = sum(epoch_val_acc)/len(epoch_val_acc)
        epoch_train_acc = sum(epoch_train_acc)/len(epoch_train_acc)
        
        print('Epoch', i, '-- Train Loss', epoch_train_loss, '-- Train MSE', epoch_train_acc, 
              '-- Val Loss', epoch_val_loss, '-- Val MSE', epoch_val_acc)

        history['train_acc'].append(epoch_train_acc)
        history['train_loss'].append(epoch_train_loss)
        history['val_acc'].append(epoch_val_acc)
        history['val_loss'].append(epoch_val_loss)
    
    return model, history

In [17]:
trained_model, history = train(model, train_set, val_set, num_epochs=10, batch_size=100)

Epoch 0 -- Train Loss 232228.3659206081 -- Train MSE 232228.3635838964 -- Val Loss 206486.631238759 -- Val MSE 206486.63399280576
Epoch 1 -- Train Loss 220967.61469594596 -- Train MSE 220967.6143018018 -- Val Loss 206048.83503821943 -- Val MSE 206048.82924910073
Epoch 2 -- Train Loss 220937.29766328828 -- Train MSE 220937.2940456081 -- Val Loss 206094.7419064748 -- Val MSE 206094.73223920862
Epoch 3 -- Train Loss 220791.6094313063 -- Train MSE 220791.609740991 -- Val Loss 206297.77439298562 -- Val MSE 206297.7664118705
Epoch 4 -- Train Loss 220615.19590371623 -- Train MSE 220615.19752252253 -- Val Loss 206713.78271133095 -- Val MSE 206713.7872077338
Epoch 5 -- Train Loss 220474.55710867117 -- Train MSE 220474.5534206081 -- Val Loss 206965.37055980216 -- Val MSE 206965.37027877697
Epoch 6 -- Train Loss 220368.3245777027 -- Train MSE 220368.32159346846 -- Val Loss 207379.32121178057 -- Val MSE 207379.31677158273
Epoch 7 -- Train Loss 220355.8691019144 -- Train MSE 220355.8647240991 -- Va

In [18]:
y_pred, _ = model(x_test)

In [1]:
plt.scatter(y_test[:,5], y_pred.detach().numpy()[:,5])

NameError: ignored